In [6]:
import pandas as pd
import numpy as np
don = pd.read_csv("C:/Users/cepe-s4-03/Documents/GB/data/ozonecomplet.csv", header=0, sep=";")
don = don.drop(['nomligne', 'Ne', 'Dv'], axis=1)
don.describe()
don.rename(columns={'O3': 'Y'}, inplace=True)
don.describe()

,Y,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,O3v
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,90.303571,18.360714,21.526786,22.627679,4.928571,5.017857,4.830357,-1.214346,-1.611004,-1.690683,90.571429
std,28.187225,3.122726,4.042321,4.530859,2.594916,2.281860,2.332259,2.632742,2.795673,2.810198,28.276853
min,42.000000,11.300000,14.000000,14.900000,0.000000,0.000000,0.000000,-7.878500,-7.878500,-9.000000,42.000000
25%,70.750000,16.200000,18.600000,19.275000,3.000000,4.000000,3.000000,-3.276450,-3.564700,-3.939200,71.000000
50%,81.500000,17.800000,20.550000,22.050000,6.000000,5.000000,5.000000,-0.866000,-1.879400,-1.549650,82.500000
75%,106.000000,19.925000,23.550000,25.400000,7.000000,7.000000,7.000000,0.694600,0.000000,0.000000,106.000000
max,166.000000,27.000000,33.500000,35.500000,8.000000,8.000000,8.000000,5.196200,6.577800,5.000000,166.000000


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import KFold, GridSearchCV

blabla

In [8]:
np.random.seed(1234)
nb=4
tmp = np.arange(don.shape[0])%nb
print(pd.Series(tmp).value_counts())
bloc = np.random.choice(tmp,size=don.shape[0],replace=False)
print(bloc)

don["bloc"] = bloc
Cible = pd.DataFrame(don[["bloc","Y"]])
PREV = pd.DataFrame({'bloc':don['bloc'],'Y':don['Y'],'MCO':0.0,'ridge':0.0,'lasso':0.0,'elas':0.0,'arbre':0.0,'foret':0.0})

0    28
1    28
2    28
3    28
Name: count, dtype: int64
[2 2 2 2 2 1 0 1 1 3 0 1 3 3 3 0 0 1 3 1 1 2 0 0 3 3 1 0 1 3 0 1 1 0 0 3 2
 1 2 0 2 2 3 1 1 1 0 3 3 2 1 0 0 1 2 0 0 3 2 3 2 0 2 0 0 1 0 1 3 2 1 0 2 0
 3 2 2 1 0 3 2 3 2 3 0 3 3 1 2 1 3 2 3 1 0 1 0 2 2 1 3 2 2 3 1 3 0 0 1 2 3
 3]


In [9]:
for i in np.arange(nb):
    print(i)
    X_train = don[don["bloc"]!=i].drop(["Y","bloc"],axis=1)
    X_test = don[don["bloc"]==i].drop(["Y","bloc"],axis=1)
    Y_train = Cible[Cible["bloc"]!=i].drop("bloc",axis=1)
    #### reg
    reg = LinearRegression()
    reg.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'MCO'] = reg.predict(X_test)
    #### pipeline
    kf = KFold(n_splits=10,shuffle=True)
    cr = StandardScaler()
    #### lasso
    lassocv = LassoCV(cv=kf)
    pipelassocv = Pipeline(steps=[("cr",cr),("lassocv",lassocv)])
    etape_lassocv = pipelassocv.named_steps["lassocv"]
    pipelassocv.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'lasso'] = pipelassocv.predict(X_test)
    ##### ridge
    grilleridge = etape_lassocv.alphas_ * 100 
    ridgecv = RidgeCV(cv=kf,alphas=grilleridge)
    piperidgecv = Pipeline(steps=[("cr",cr),("ridgecv",ridgecv)])
    piperidgecv.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'ridge'] = piperidgecv.predict(X_test)
    ##### elasticnet
    grilleelas = etape_lassocv.alphas_ * 2 
    elasticcv = ElasticNetCV(cv=kf,alphas=grilleelas)
    pipeelasticcv = Pipeline(steps=[("cr",cr),("elasticcv",elasticcv)])
    pipeelasticcv.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'elas'] = pipeelasticcv.predict(X_test)
    #### reg
    arbre = DecisionTreeRegressor()
    arbre.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'arbre'] = arbre.predict(X_test)
    #### foret
    foret = RandomForestRegressor(n_estimators=500)
    foret.fit(X_train,Y_train)
    PREV.loc[PREV.bloc==i,'foret'] = foret.predict(X_test)

0


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cepe-s4-03\AppData\Local\Temp\ipykernel_13808\2525805548.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  foret.fit(X_train,Y_train)


1


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cepe-s4-03\AppData\Local\Temp\ipykernel_13808\2525805548.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  foret.fit(X_train,Y_train)


2


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cepe-s4-03\AppData\Local\Temp\ipykernel_13808\2525805548.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  foret.fit(X_train,Y_train)


3


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cepe-s4-03\AppData\Local\Temp\ipykernel_13808\2525805548.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  foret.fit(X_train,Y_train)


In [10]:
Erreur = PREV.copy()
Erreur = Erreur.drop("bloc",axis=1)


def erreur(X, Y):
    return np.mean((X - Y) ** 2)

def apply_erreur(RES):
    return RES.apply(lambda col: erreur(col, RES.iloc[:, 0]), axis=0)

apply_erreur(Erreur)

Y          0.000000
MCO      246.612331
ridge    231.011861
lasso    226.719158
elas     225.578860
arbre    483.687500
foret    255.549443
dtype: float64